In [1]:
import os
import nltk
import numpy as np
import math
from nltk.corpus import stopwords
import docx
from docx import Document


#doc file input in python
def getText(filename):
    documentation = Document(filename)
    fulltext=[]
    for parag in documentation.paragraphs:
        fulltext.append(parag.text)
    print("\n ".join(fulltext))
    return "\n".join(fulltext)
#input filepath where all assignment belong
docFiles=[]
for filename in os.listdir(r"C:\Users\Sudheer\GIT\BMSP"):
    if (not filename.startswith('~')) & filename.endswith('.docx') :
        getText(filename)
        docFiles.append(filename)
print(docFiles)
docFiles.sort(key=str.lower)
print(len(docFiles))




 1.Unit Level Testing:-
 --------------------
 A smallest testing part(or)portion in the same code of the application is called as "Unit testing".
 --->Each and every part of unit testing working(or)not
 -->Unit Testing is also known as "Module Testing".
1.Top-Down approach:-
 -----------------------
 This approach is recommened if there are any incomplete program at bottom level,in this approach integration testing will be carried out from top to bottom,the incomplete will be replaced with "STUB".
3.Hybrid Approach :-
 ------------------
 It is a combination of Top-down and bottom-up approach.
 -->The incomplete program will be replaced botH STUB an Driver.
Blackbox Testing:-
 -----------------
 Once 100% is coding is completed and white box completed to confirm correctness of requirement on the application is called as "Balck Box testing".
 
 --->This testing is conducted by testing team/testers.
 
 --->In Blackbox it is divided into 2 Levels.
 1.System Level Testing
 2.UAT (User Ac

In [2]:
#building vocabulary of all the docuents
def build_lexicon(corpus):
    lexicon=set()
    for doc in corpus:
        #word tokenization
        word_token=[word for word in doc.split()]
        lower_word_list=[i.lower() for i in word_token]
        #stemming
        porter = nltk.PorterStemmer()
        stemmed_word=[porter.stem(t) for t in lower_word_list]
        #removing stop words
        stop_words = set(stopwords.words('english'))
        filtered_bag_of_word=[w for w in stemmed_word if not w in stop_words]
        lexicon.update(filtered_bag_of_word)
    return lexicon

In [3]:
vocabulary=build_lexicon(docFiles)

In [4]:
def tf(term, document):
    return freq(term, document)
def freq(term, document):
    return document.split().count(term)

In [5]:
doc_term_matrix=[]
print("\n Our Vocabulary vector is ["+','.join(list(vocabulary)) + "] ")
for doc in docFiles:
    tf_vector=[tf(word, doc) for word in vocabulary]
    tf_vector_string=",".join(format (freq, 'd') for freq in tf_vector)
    print("\n the tf vector for document %d is [%s]" % ((docFiles.index(doc)+1),tf_vector_string))
    doc_term_matrix.append(tf_vector)
print("\n All combined here is our master document term matrix:")
print(doc_term_matrix)


 Our Vocabulary vector is [5.docx,3.docx,1.docx,4.docx,2.docx] 

 the tf vector for document 1 is [0,0,1,0,0]

 the tf vector for document 2 is [0,0,0,0,1]

 the tf vector for document 3 is [0,1,0,0,0]

 the tf vector for document 4 is [0,0,0,1,0]

 the tf vector for document 5 is [1,0,0,0,0]

 All combined here is our master document term matrix:
[[0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0]]


#Now every document is in the same feature space
#Normalizing vectors to  norm
#l2 norm of each vector is 1

In [6]:
def l2_normalizer(vec):
    denom = np.sum([el**2 for el in vec])
    if math.sqrt(denom):
        return [(el/math.sqrt(denom)) for el in vec]
    else:
        return vec

In [7]:
doc_term_matrix_l2=[]
for vec in doc_term_matrix:
    doc_term_matrix_l2.append(l2_normalizer (vec))
print("\n A regular old document term matrix: ")
print(np.matrix(doc_term_matrix))
print("\nA document term matrix with row wise l2 norms of 1: ")
print(np.matrix(doc_term_matrix_l2))


 A regular old document term matrix: 
[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]]

A document term matrix with row wise l2 norms of 1: 
[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


In [8]:
def numDocsContaining (word, doclist):
    doccount =0
    for doc in doclist:
        if freq(word, doc)>0:
            doccount +1
    return doccount

In [9]:
def idf (word,doclist):
    n_samples=len(doclist)
    df= numDocsContaining (word, doclist)
    return np.log(n_samples/1+df)

In [10]:
my_idf_vector=[idf(word, docFiles) for word in vocabulary]
print("our vocabuary vector is[ "+ ",".join(list (vocabulary)) + "]")

our vocabuary vector is[ 5.docx,3.docx,1.docx,4.docx,2.docx]


In [11]:
def build_idf_matrix(idf_vector):
    idf_mat=np.zeros((len(idf_vector), len(idf_vector)))
    np.fill_diagonal(idf_mat,idf_vector)
    return idf_mat

In [12]:
my_idf_matrix=build_idf_matrix(my_idf_vector)
print("\nIdf matrix is:\n",my_idf_matrix)


Idf matrix is:
 [[1.60943791 0.         0.         0.         0.        ]
 [0.         1.60943791 0.         0.         0.        ]
 [0.         0.         1.60943791 0.         0.        ]
 [0.         0.         0.         1.60943791 0.        ]
 [0.         0.         0.         0.         1.60943791]]


In [13]:
doc_term_matrix_tfidf=[]
#performing tfidf matrix multiplication
for tf_vector in doc_term_matrix:
    doc_term_matrix_tfidf.append(np.dot(tf_vector,my_idf_matrix))

In [14]:
doc_term_matrix_tfidf_l2=[]
for tf_vector in doc_term_matrix_tfidf:
    doc_term_matrix_tfidf_l2.append(l2_normalizer(tf_vector))
print(vocabulary)
print(np.matrix(doc_term_matrix_tfidf_l2))

{'5.docx', '3.docx', '1.docx', '4.docx', '2.docx'}
[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


In [15]:
for i in range(len(docFiles)):
    for j in range(i+1, len(docFiles)):
        result_nltk=nltk.cluster.util.cosine_distance(doc_term_matrix_tfidf_l2[i],doc_term_matrix_tfidf_l2[j])
        print("\n Cosine Distance btw doc %d and doc %d:" %(i,j))
        print (result_nltk)
        cos_sin=1-result_nltk
        try:
            angle_in_radians=math.acos(cos_sin)
        except ValueError:
            print("Here Error")
        if not math.isnan(cos_sin):
            plagiarism=int(cos_sin*100)
            print("\nPlagiarism =%s" % plagiarism)
        else:
            print("\n Plagiarism = 1")


 Cosine Distance btw doc 0 and doc 1:
1.0

Plagiarism =0

 Cosine Distance btw doc 0 and doc 2:
1.0

Plagiarism =0

 Cosine Distance btw doc 0 and doc 3:
1.0

Plagiarism =0

 Cosine Distance btw doc 0 and doc 4:
1.0

Plagiarism =0

 Cosine Distance btw doc 1 and doc 2:
1.0

Plagiarism =0

 Cosine Distance btw doc 1 and doc 3:
1.0

Plagiarism =0

 Cosine Distance btw doc 1 and doc 4:
1.0

Plagiarism =0

 Cosine Distance btw doc 2 and doc 3:
1.0

Plagiarism =0

 Cosine Distance btw doc 2 and doc 4:
1.0

Plagiarism =0

 Cosine Distance btw doc 3 and doc 4:
1.0

Plagiarism =0
